In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from transformers import RobertaTokenizer, RobertaModel,  RobertaForSequenceClassification

In [2]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [22]:
robertamodel =RobertaModel.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
ref = ["i hate you","i love you","i have an apple",
      "a man is doing his task","an woman is doing his task","what fuck!",
      "All is best","we belong to each other","an apple a day, keep a doctor away",
       "it is a test to unbanance the size of two corpus"
      ]
ref_input_ids = pad_sequence([torch.tensor(tokenizer.encode(item))  for item in ref],batch_first=True,
                         padding_value=tokenizer.pad_token_id)
print(ref_input_ids)

tensor([[    0,   118,  4157,    47,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,   118,   657,    47,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,   118,    33,    41, 15162,     2,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,   102,   313,    16,   608,    39,  3685,     2,     1,     1,
             1,     1,     1,     1,     1],
        [    0,   260,   693,    16,   608,    39,  3685,     2,     1,     1,
             1,     1,     1,     1,     1],
        [    0, 12196, 26536,   328,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,  3684,    16,   275,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,  1694,  9943,     7,   349,    97,     2,     1,     1,     1,
             1,     1,     1,     1,     1],
        

In [77]:
print(ref_input_ids.dtype)

torch.int64


In [75]:
ref_output =robertamodel(ref_input_ids)

In [51]:
print(ref_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0231,  0.0858, -0.0508,  ..., -0.1480, -0.0463, -0.0396],
         [ 0.0033, -0.1636, -0.2311,  ...,  0.0705, -0.0411, -0.0622],
         [ 0.0919,  0.0255,  0.1190,  ...,  0.1001, -0.0074, -0.2072],
         ...,
         [ 0.0373,  0.0792, -0.1062,  ..., -0.4763,  0.0509, -0.0147],
         [ 0.0373,  0.0792, -0.1062,  ..., -0.4763,  0.0509, -0.0147],
         [ 0.0373,  0.0792, -0.1062,  ..., -0.4763,  0.0509, -0.0147]],

        [[-0.0344,  0.0929, -0.0547,  ..., -0.1331, -0.0475, -0.0297],
         [-0.0433, -0.1727, -0.1718,  ...,  0.0806, -0.0417,  0.0471],
         [ 0.0448,  0.0110,  0.1917,  ...,  0.1734, -0.0259, -0.0411],
         ...,
         [ 0.0200,  0.0745, -0.1042,  ..., -0.4673, -0.0379,  0.0738],
         [ 0.0200,  0.0745, -0.1042,  ..., -0.4673, -0.0379,  0.0738],
         [ 0.0200,  0.0745, -0.1042,  ..., -0.4673, -0.0379,  0.0738]],

        [[-0.0425,  0.0870, -0.0404,  ..., -0.1532, -

In [61]:
l1 = nn.Linear(768,64)
active_l1 = F.relu
l2 = nn.Linear(64,1)
active_l2 = F.sigmoid

In [62]:
out = active_l2 (l2(active_l1(l1(ref_output[1]))))

/home/hqh/.conda/envs/torch1.10/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [63]:
print(out)

tensor([[0.4871],
        [0.4868],
        [0.4873],
        [0.4873],
        [0.4866],
        [0.4870],
        [0.4865],
        [0.4889],
        [0.4873],
        [0.4886]], grad_fn=<SigmoidBackward0>)


In [64]:
loss = torch.log(out).mean()
print(loss)

tensor(-0.7188, grad_fn=<MeanBackward0>)


In [71]:
c = nn.BCELoss()
label = torch.ones(len(out))
print(c(out.squeeze(1),label))

tensor(0.7188, grad_fn=<BinaryCrossEntropyBackward0>)


In [79]:
classfication = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels = 3)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [83]:
ref_output =classfication(ref_input_ids)

In [85]:
print(ref_output.logits)

tensor([[ 0.0502, -0.0635,  0.0252],
        [ 0.0439, -0.0607,  0.0218],
        [ 0.0499, -0.0577,  0.0295],
        [ 0.0595, -0.0695,  0.0293],
        [ 0.0548, -0.0642,  0.0225],
        [ 0.0517, -0.0749,  0.0349],
        [ 0.0443, -0.0631,  0.0227],
        [ 0.0625, -0.0632,  0.0319],
        [ 0.0733, -0.0698,  0.0247],
        [ 0.0703, -0.0631,  0.0451]], grad_fn=<AddmmBackward0>)


In [90]:
print(torch.argmax(ref_output.logits,dim=-1))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [97]:
x = torch.tensor([[0,1,0],[1,0,0]])
y = torch.argmax(x,dim=-1)
x_hat = torch.tensor([[0.8,0.1,0.1],[0.1,0.8,0.1]])
y_hat = torch.argmax(x_hat,dim=-1)
acc = (y==y_hat).sum()
print(acc)

tensor(0)


In [101]:
'sad'+str(acc.item())

'sad0'